In [1]:
%%HTML
<style>
    body {
        --vscode-font-family: "KoddiUD 온고딕"
    }
</style>

# 코호트 분석
- 기준
  - 시간 기반 코호트
  - 행동 기반 코호트
  - 인구 통계학적 코호트

## ⏱️ 시간 기반 코호트

In [10]:
import pandas as pd
import numpy as np
import random

In [7]:
# 가상 {데이터 생성
data = {
  'customer_id' : range(1, 501),
  'signup_date' : random.sample(pd.date_range(start='2023-01-01', periods=500, freq='D').tolist(), 500),
  'total_payment' : np.random.randint(100, 500, 500)
}

In [15]:
df = pd.DataFrame(data)
df.head()

,customer_id,signup_date,total_payment
0,1,2024-02-23,244
1,2,2023-10-18,288
2,3,2023-03-27,451
3,4,2023-05-27,332
4,5,2024-03-01,219


In [16]:
df['signup_based_cohort'] = df['signup_date'].dt.to_period('M')
df.head()

,customer_id,signup_date,total_payment,signup_based_cohort
0,1,2024-02-23,244,2024-02
1,2,2023-10-18,288,2023-10
2,3,2023-03-27,451,2023-03
3,4,2023-05-27,332,2023-05
4,5,2024-03-01,219,2024-03


## 🏃 행동 기반 코호트

e.g. 첫 구매 상품

In [23]:
# 가상 데이터 생성
data = {
  'customer_id':np.random.randint(1, 100, 500),
  'order_date': random.sample(pd.date_range(start='2023-01-01', periods = 500, freq='D').tolist(), 500),
  'product_id': [np.random.choice([101, 102, 103, 104, 105])for i in range(500)],
  'total_payment': np.random.randint(100, 500, 500)
}

In [24]:
pd.DataFrame(data)
df.head()

,customer_id,signup_date,total_payment,signup_based_cohort
0,1,2024-02-23,244,2024-02
1,2,2023-10-18,288,2023-10
2,3,2023-03-27,451,2023-03
3,4,2023-05-27,332,2023-05
4,5,2024-03-01,219,2024-03


In [34]:
# 고객별로 첫 구매 날짜 찾기
first_purchase = df.groupby('customer_id')['order_date'].min().reset_index()
first_purchase.columns = ['customer_id', 'first_purchase_date']
first_purchase

,customer_id,first_purchase_date
0,1,2023-05-15
1,2,2023-04-20
2,3,2023-02-27
3,4,2023-02-04
4,5,2023-01-31
...,...,...
93,95,2023-09-27
94,96,2023-02-05
95,97,2023-01-17
96,98,2023-01-14


In [35]:
# 고객 별 첫 구매 날짜와 고객 ID를 사용하여 첫 구매 상품 찾기
first_purchase.head(3)

,customer_id,first_purchase_date
0,1,2023-05-15
1,2,2023-04-20
2,3,2023-02-27


In [36]:
df.head()

,customer_id,order_date,product_id,total_payment
0,62,2023-07-18,101,353
1,24,2024-03-23,104,320
2,92,2024-01-07,101,276
3,84,2023-04-29,101,377
4,11,2024-05-06,102,283


In [39]:
first_product = first_purchase.merge(df, 
                      left_on = ['customer_id','first_purchase_date'],
                      right_on = ['customer_id', 'order_date'],
                      how = 'left')
first_product.head()

,customer_id,first_purchase_date,order_date,product_id,total_payment
0,1,2023-05-15,2023-05-15,103,160
1,2,2023-04-20,2023-04-20,105,208
2,3,2023-02-27,2023-02-27,102,127
3,4,2023-02-04,2023-02-04,102,132
4,5,2023-01-31,2023-01-31,101,484


In [40]:
first_product = first_product[['customer_id', 'product_id']]
first_product.columns = ['customer_id', 'first_product_cohort']

In [41]:
first_product.head(10)

,customer_id,first_product_cohort
0,1,103
1,2,105
2,3,102
3,4,102
4,5,101
5,6,101
6,7,102
7,8,103
8,9,105
9,10,102


## 🧑‍🧑‍🧒‍🧒 인구 통계학적 코호트

In [42]:
# 가상 데이터 생성
data = {
    'customer_id' : range(1, 101),
    'age' : np.random.randint(18, 70, 100),
    'gender' : np.random.choice(['Male', 'Female'], 100),
    'total_payment' : np.random.randint(100, 1000, 100)
}

In [43]:
df = pd.DataFrame(data)
df.head()

,customer_id,age,gender,total_payment
0,1,63,Female,381
1,2,45,Male,601
2,3,18,Female,149
3,4,39,Male,300
4,5,41,Female,845


In [44]:
df['age_group'] = pd.cut(df['age'], bins=[0, 20, 30, 40, 50, 60, 70], labels=['10s', '20s', '30s', '40s', '50s', '60s'])
df['cohort'] = df.apply(lambda x : f"{x['age_group']}_{x['gender']}", axis=1)
df.head()

,customer_id,age,gender,total_payment,age_group,cohort
0,1,63,Female,381,60s,60s_Female
1,2,45,Male,601,40s,40s_Male
2,3,18,Female,149,10s,10s_Female
3,4,39,Male,300,30s,30s_Male
4,5,41,Female,845,40s,40s_Female


## 코호트 분석

In [46]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [49]:
# 가상 데이터 생성
n_customers = 500
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = (end_date - start_date).days

In [51]:
data = {
  'customer_id': np.random.choice(range(1, n_customers + 1), size=1000),
  'purchase_date': [start_date + timedelta(days = np.random.randint(date_range))for _ in range(1000)],
  'purchase_amount': np.random.uniform(100, 1000, size = 1000)
}

In [52]:
# 매달 1일로 지정 -> 연도와 월 기준 코호트


### 코호트 정의

In [53]:
df['cohort'] = df['purchase_date'].apply(lambda x: x.replavce(day=1))

df.head()

KeyError: 'purchase_date'

In [54]:
# 각 고객의 첫 구매 날짜 할당
df.groupby('customer_id')['purchase_date'].min().apply(lambda x: x.replace(day=1))

KeyError: 'Column not found: purchase_date'

In [ ]:
# 코호트별 월별 지표 계산
df['month_since_first_purchase'] = df['cohort'].dt.year - df['first_purchase_date'].dt.year) * 12 + (df['cohort'].dt.month - df['first_purchase_date'].dt.month)

SyntaxError: unmatched ')' (4106786434.py, line 2)

In [58]:
cohort_data = df.groupby(['first_purchase_date', 'month_since_first_purchase'].agg(n_customers = ['customer_id', 'nunique'], total_purchases('purchase_amount', 'sum')).reset_index()

SyntaxError: positional argument follows keyword argument (4175803987.py, line 1)

group by에 넣는 값은 첫 구매 날짜, 며칠이나 지났는지

In [59]:
cohort_data.head()

NameError: name 'cohort_data' is not defined

**피벗 테이블** 만들기

In [61]:
# 코호트 피벗 테이블
cohort_pivot = cohort_data.pivot_table(index = 'first_purchase_date', columns)

SyntaxError: positional argument follows keyword argument (2767586053.py, line 2)

In [62]:
# 코호트 사이즈 산출
cohort_pivot.illoc[:0]

NameError: name 'cohort_pivot' is not defined

**해석**

2024년 월별로 코호트(첫 구매 월)에 속한 코호트 사이즈(사용자 수)로 해당 월에 첫 구매를 한 사용자의 수

- 초기 구매자 수가 높고 이후 감소 추세
  - 2024년 1월(87명) 이후 사용자 수가 점차 감소하며, 9월부터는 30명 이하로 감소 추세
  - 시간이 지남에 따라 첫 구매를 하는 사용자가 줄어들고 있음을 의미하며, 서비스에 대한 관심이나 구매 전환율이 초기보다는 감소하고 있을 가능성을 시사
- 시즌별 구매 변화 가능성
  - 특정 월에 구매자가 크게 증가하는 패턴이 보지디 않는 것으로 보아 계절적 요인이나 프로모션에 따른 성장은 확인되지 않음
- 결론적으로 시간이 지남에 따라 신규 사용자 유입이 줄어드는 문제가 있음
  - 초기 유입과 첫 구매 전환율을 높이는 전략 필요
  - 첫 구매 전환율을 높이는 전략 필요


In [ ]:
# 유지율 계산
retention_matrix = np.round(cohort_pivot.divide(cohort_size, axis =0) * 100, 2)
retention_matrix

NameError: name 'cohort_pivot' is not defined

In [64]:
# 대체로 감소 추세
# 코호트 추세가 안정적 -> 어떤 이유로 장기 충성을 보이는 지 알아보기

**해석**
- 월별로 첫 구매를 한 사용자 코호트에 대해서 각 코호트가 몇 개월 후에도 활동(구매)를 지속하는 비율을 나타내는 리텐션 분석 결과, 각 값은 해당 코호트에서 첫 구매 후 특정 개월 수가 지난 시점까지 남아 있는(다시 구매한) 사용자의 비율
  - e.g. 2024년 1월에 첫 구매를 한 코호트의 1개월 차 리텐션: 10.34%